# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [8]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [9]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [10]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

In [11]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [12]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [13]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [14]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [15]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [16]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [17]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [18]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [19]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [20]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [114]:
!cat .env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [102]:
filename='Acessos-ERS-240 OK'
road_name = '240'

Escolhendo a aba da planilha do Excel

In [103]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [104]:
print(ws['A1'].value)

km


Carregando os valores no script

In [105]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            obj['img'][0].name='Verso'
        except:
            pass

        try:
            obj['img'][1].name='Frente'
        except:
            pass

        
        
    values.append(obj)

Teste de valores importados da planilha

In [106]:
len(values),values[0]

(198,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': -29723810,
  'Longitude_Inicial': -51168591,
  'Latitude_Final': None,
  'Longitude_Final': None,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Decrescente',
  'Classe': 'Acessos',
  'Faixa': None,
  'Rodovia': 'ERS-240',
  'Tipo de Acesso': '1 - ACI E',
  'Observações': None,
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': None,
  'Foto_2': None,
  'Data Foto_2': None,
  'Tipo Foto_2': None,
  'Descrição Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4': None,
  'Tipo Foto_4': None,
  'Descrição Foto_4': None,
  'Foto_5': None,
  'Data Foto_5': None,
  'Tipo Foto_5': None,
  'Descrição Foto_5': None,
  'Foto_6': None,
  'Data Foto_6': None,
  'Tipo Foto_6'

Definindo valores da importação

In [107]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Acessos
occurrence_type=OccurrenceType.objects.get(uuid='c7faa314-2bf3-43ed-8952-117153b82330')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
company,occurrence_type,firm,road_name

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Acessos - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>,
 '240')

In [108]:
for a in values:
    a['Longitude_Inicial'] = str(a['Longitude_Inicial']).replace(',','').replace('.','')
    a['Latitude_Inicial'] = str(a['Latitude_Inicial']).replace(',','').replace('.','')

Criando itens de Inventário

In [109]:
objects=[]


for index, a in enumerate(tqdm(values)):
    long=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]
    lat=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
    road,_,point,km =point_to_km(float(long),float(lat),road_name)
    if road_name not in road.name:
        print(f"Rodovia {road} - Km:{km} - Linha: {index+2}")

    try:

        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'] if a['Faixa'] is not None else 'Não se aplica')),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
#                     end_km=float(str(a['km final']).replace(',','.')),
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'type': a['Tipo de Acesso'],                                                     
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/198 [00:00<?, ?it/s]

ValueError: could not convert string to float: 'Non.e'

Salvando itens de Inventário no sistema e imprimindo os seriais

In [118]:
objects[178][0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f7f95d84e10>,
 'uuid': UUID('c188e81e-ad61-4402-86cd-a15bbfba51ab'),
 'number': '',
 'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
 'road_name': 'ERS-240',
 'road_id': 2060,
 'km': 31.694,
 'end_km': None,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f7f96c86490>,
 'direction': '0',
 'lane': '99',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('0edd2ecc-8d5e-4444-898b-0069a3e9c354'),
 'occurrence_type_id': UUID('c7faa314-2bf3-43ed-8952-117153b82330'),
 'form_data': {'type': '179 - ARU - D'},
 'form_metadata': {},
 'executed_at': None,
 'created_at': None,
 'updated_at': None,
 'found_at': datetime.datetime(2023, 8, 3, 21, 2, 43, 173583, tzinfo=<UTC>),
 'due_at': None,
 'status_id': UUID('c4b0df55-dc0e-47ba-83e7-86080047a8d5'),
 'job_id': None,
 'parent_id': None,
 'active_inspection_id': No

In [119]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/179 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27129
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27130
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27131
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27132
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27133
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27134
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27135
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27136
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CS

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27202
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27203
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27204
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27205
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27206
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27207
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27208
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27209
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CS

CSG-Inv-2023.27275
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27276
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27277
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27278
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27279
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27280
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27282
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2060: ERS-240
CSG-Inv-2023.27283
Searching KM on direction...
Searching KM without direction...
Found KM on Ro

Anexando as imagens